# rdrp Benchmark / GITB
```
Lead     : ababaian /rce
Issue    : 
start    : 2020 12 30
complete : 2021 01 01
files    : ~/serratus/notebook/20123_gitb/
s3 files : s3://serratus-public/notebook/201230_gitb/
```

## Introduction

> `rce:`
> For validation of RdRp identification and trimming, we should do hold-out cross-validation using high-quality sequences, i.e. RefSeqs. To assess how accuracy varies with identity, we will use cross-validation by identity (CVI, https://www.drive5.com/usearch/manual/cvi.html) which constructs test/training sets where the maximum identity (MaxID) of the top test-training hit is fixed to a known value or range of values. Here, I suggest five test/training pairs with MaxID = 90, 75, 50, 30, 15% to get a representative range.
>
> The gold standard reference for the gene identification and trimming benchmark (GITB-gold) will be full-length RdRp-containing ORFs from RefSeqs plus coordinates of the trimmed RdRps. These coordinates will be obtained by aligning the ORFs to Wolf18. These alignments should be good (diamond may be good enough) because the top-hit identities should be at least 90%.
>
> The decoy reference (GITB-decoy) will be all RefSeq ORFs which do NOT contain RdRp. Ideally, these will all be discarded by the protocol. In practice, there will be FPs due to local homology between RdRp and other genes; this surely arises often in viruses due to recombination events. Using the decoy reference will enable us to design better filters and measure the FP rate (ROC curve) of the protocol.

Most of the code for this was developed already in `201210_RdRp_panproteome_v1.ipyn` notebook. This is a self-contained migration of that code / clean-up.

In [1]:
# date and version
date

Wed Dec 30 15:25:39 PST 2020


In [ ]:
# Upload gitb folder (from EC2)
aws s3 sync ./ s3://serratus-public/notebook/201230_gitb/

## EC2 Benchmark environment

- OS: `Amazon Linux 2 AMI (HVM) x86`
- ami: `ami-0be2609ba883822ec`
- instance: `c5.xlarge`
- description: `"c5.xlarge (- ECUs, 4 vCPUs, 3.4 GHz, -, 8 GiB memory, EBS only)"`
- storage: `12 GiB SSD (gp2)`
- encryption: `false`


In [ ]:
# From base amazon linux 2
sudo yum install -y docker
sudo yum install -y git
sudo service docker start


# Launch "Serratus-Production" container environment
git clone -b diamond-dev https://github.com/ababaian/serratus.git
cd serratus/containers

# Build `serratus-align` container
# NOTE: diamond v0.9.35.136 Installed
./build_containers.sh
sudo docker run --rm --entrypoint /bin/bash -it serratus-align:latest

In [ ]:
# From `serratus-align` container

# Install diamond2 
# Libraries for building diamond2
yum -y install git gcc gcc-c++ glibc-devel \
  cmake patch automake zlib-devel make

# grab latest with fix from Benjamin
git clone https://github.com/bbuchfink/diamond.git
cd diamond

mkdir bin; cd bin
cmake ..
make -j4
cp ./diamond /usr/bin/diamond2
#sudo make install

In [ ]:
# From `serratus-align` container
mkdir /home/serratus/test; cd /home/serratus/test

# Login to your AWS account
# aws configure
# region: us-east-1

# Download rdrp test-database and test-data
curl https://serratus-public.s3.amazonaws.com/notebook/201226_rdrp0/rev1/rdrp0_r1.fa > rdrp0.fa
curl https://serratus-public.s3.amazonaws.com/test-data/fq-blocks/ERR2756788.fq.00 > ERR2756788.fq.00
curl https://serratus-public.s3.amazonaws.com/test-data/fq-blocks/ERR2756788.fq.01 > ERR2756788.fq.01
curl https://serratus-public.s3.amazonaws.com/test-data/fq-blocks/ERR2756788.fq.02 > ERR2756788.fq.02
curl https://serratus-public.s3.amazonaws.com/test-data/fq-blocks/ERR2756788.fq.03 > ERR2756788.fq.03

# diamond1 database
diamond makedb  --in rdrp0.fa -d rdrp0.v1
diamond2 makedb --in rdrp0.fa -d rdrp0.v2

In [ ]:
# NOTE: diamond v0.9.35.136
# This amounts to ~ $0.02 / library at production scale
run_diamond () {
    QUERY="$1"
    RDRP="$2"
    OUTNAME="$3"
    
    # Diamond blastp "SERRATUS PRODUCTION CODE"
    time cat $QUERY |\
    diamond blastx \
        -d "$RDRP".dmnd \
        --unal 0 \
        -k 1 \
        -p 1 \
        -b 0.4 \
        -f 6 qseqid sseqid qstart qend qlen sstart send slen pident evalue btop cigar qstrand full_qseq sseq \
        > "$OUTNAME".pro

}

run_set () {
    run_diamond ERR2756788.fq.00 rdrp0.v1 b0.v1 & 
    run_diamond ERR2756788.fq.01 rdrp0.v1 b1.v1 &
    run_diamond ERR2756788.fq.02 rdrp0.v1 b2.v1 &
    run_diamond ERR2756788.fq.03 rdrp0.v1 b3.v1 &
    
    echo DONE
}

run_set

# real    3m15.520s
# user    3m10.552s
# sys     0m4.847s

# real    3m15.818s
# user    6m21.440s
# sys     0m9.486s

# real    3m15.819s
# user    6m21.440s
# sys     0m9.486s

# real    3m16.375s
# user    3m10.927s
# sys     0m4.941s

In [ ]:
# Diamond2 with Buchfunk optimizations

run_diamond2 () {
    QUERY="$1"
    RDRP="$2"
    OUTNAME="$3"
    
    # Diamond blastp alignment "BUCHFINK OPTIMIZED CODE"
    time cat $QUERY |\
    diamond2 blastx \
        -d "$RDRP".dmnd \
        -c1 -p1 -k1 \
        --more-sensitive \
        --masking 0 \
        --target-indexed \
        > "$OUTNAME".pro
#         --gapped-filter-evalue 0 \          
}

run_diamond2 ERR2756788.fq.00 rdrp0.v2 b0.v2
# real    3m54.125s
# user    3m45.035s
# sys     0m7.377s

run_set2 () {
    run_diamond2 ERR2756788.fq.00 rdrp0.v2 b0.v2 & 
    run_diamond2 ERR2756788.fq.01 rdrp0.v2 b1.v2 &
    run_diamond2 ERR2756788.fq.02 rdrp0.v2 b2.v2 &
    run_diamond2 ERR2756788.fq.03 rdrp0.v2 b3.v2 &
    
    echo DONE
}

run_set2

# [2]   Exit 137                run_diamond2 ERR2756788.fq.01 rdrp0.v2 b1.v2
# real    0m43.325s
# user    0m29.316s
# sys     0m2.881s

# real    5m58.967s
# user    5m43.560s
# sys     0m4.118s

# real    6m13.805s
# user    5m53.061s
# sys     0m9.679s

# real    6m21.922s
# user    6m11.612s
# sys     0m9.805s


In [ ]:
# Diamond2 with Buchfunk optimizations and -fast

run_diamond2 () {
    QUERY="$1"
    RDRP="$2"
    OUTNAME="$3"
    
    # Diamond blastp alignment "BUCHFINK OPTIMIZED CODE"
    time cat $QUERY |\
    diamond2 blastx \
        -d "$RDRP".dmnd \
        -c1 -p1 -k1 \
        --masking 0 \
        --target-indexed \
        > "$OUTNAME".pro
#         --gapped-filter-evalue 0 \          
}

run_set2 () {
    run_diamond2 ERR2756788.fq.00 rdrp0.v2 b0.v3 & 
    run_diamond2 ERR2756788.fq.01 rdrp0.v2 b1.v3 &
    run_diamond2 ERR2756788.fq.02 rdrp0.v2 b2.v3 &
    run_diamond2 ERR2756788.fq.03 rdrp0.v2 b3.v3 &
    
    echo DONE
}

run_set2

# CRASH
#real    0m26.172s
#user    0m22.607s
#sys     0m2.757s

#real    0m46.678s
#user    0m41.974s
#sys     0m3.417s

#real    0m53.690s
#user    0m49.022s
#sys     0m3.666s

#real    0m53.701s
#user    0m49.502s
#sys     0m3.401s



## Key Files

#### wolf: trimmed set of RdRp true positives

- `wolf18.fa` : GenBank rdrp records from wolf18
- `wolf20.fa` : Yangshen assembled records from wolf20. Exclusive to wolf18.

```
# Header fields:
# rdrp_branch.viral_family.viral_name:accession
```

#### RefSeq Curated Viral Sequences

- `vrs240.fa` : Viral RefSeq 240 Sequences (aa)
- `vrs240_w18.fa` : ALL RefSeq Sequences with a wolf18 match (includes low confidence)
- `vrs240_w18.pro` : Diamond TSV output of RefSeq vs. wolf18

#### `./gitb` Sequences

- `gitb.fa` : High-Confidence & Full-Length RdRp+ RefSeq sequences
- `gitb.rdrp.fa` : RdRp RefSeq sequences
```
# Header fields:
# RefSeq_accession rdrp_start rdrp_end rdrp_len
```
- `gitb.pro` : Diamond output high-confidence subset of `vrs240_w18.pro` 

#### `decoy/` Sequences

- `gitb-decoy.fa` : Curated decoy proteins from human, yeast, fly, ecoli and DNA-virus proteomes
- `decoy_w18.pro` : False-positive matches of gitb-decoy.fa to wolf18.fa
- `proteome/` : individual proteomes

#### `cvi/` Cross Validation Sets (usearch)

- `cvi/`
- `gitb.all.fa` : same as ../gitb.fa
- `gitb.45.cvi`: Cross-validation set at 45% identity (40-50% sequence matchs)

## WOLF18 RdRp Header Parsing

Sequence data: `ftp://ftp.ncbi.nlm.nih.gov/pub/wolf/_suppl/rnavir18/RNAvirome.S2.afa`
Original file: `201230_gitb/wolf18/gb_rdrp.afa`
Parsed file: `201230_gitb/wolf18.fa`

The headers in this data is pre-processe

### Level 1 - Supergroup / Branches

The RdRp can be broadly classified into 5 branches which will form the lowest level of the hierarchy: `rdrp1`, `rdrp2`, `rdrp3`, `rdrp4`, `rdrp5`.

>Branch 1 consists of leviviruses and their eukaryotic relatives, namely, “mitoviruses,” “narnaviruses,” and “ourmiaviruses” (the latter three terms are placed in quotation marks as our analysis contradicts the current ICTV framework, which classifies mitoviruses and narnaviruses as members of one family, Narnaviridae, and ourmiaviruses as members of a free-floating genus, Ourmiavirus).

> Branch 2 (“picornavirus supergroup”) consists of a large assemblage of +RNA viruses of eukaryotes, in particular, those of the orders Picornavirales and Nidovirales; the families Caliciviridae, Potyviridae, Astroviridae, and Solemoviridae, a lineage of dsRNA viruses that includes partitiviruses and picobirnaviruses; and several other, smaller groups of +RNA and dsRNA viruses.

> Branch 3 consists of a distinct subset of +RNA viruses, including the “alphavirus supergroup” along with the “flavivirus supergroup,” nodaviruses, and tombusviruses; the “statovirus,” “wèivirus,” “yànvirus,” and “zhàovirus” groups; and several additional, smaller groups.

> Branch 4 consists of dsRNA viruses, including cystoviruses, reoviruses, and totiviruses and several additional families.

> Branch 5 consists of −RNA viruses.

Boundary defintions of Branches with relation to RdRp are taken from paper

Based on: [Supplementary Data 4](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6282212/bin/mbo006184203sd4.txt)
Saved as: `rdrp_representative_branches.tree`

### Level 2 - Viral Family

Based on: `https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6282212/bin/mbo006184203sd1.xls`
Saved as: `wolf18/wolf18_vlist.xlsx`

Spreadsheet includes the fields

- RdRp num ID: Ordinal numbering for RdRp
- RdRp GenBank Acc: Protein accession ID
- NCBI Tax ID: taxid from NCBI
- virus name: virus name
- taxonomy: taxonomic tree

Taxonomy field was parsed to retrieve "*dae* suffix for "Family", relatively appropriate family-name or "unclassified" when unavailable. Monkey work.

### Level 3 - Sequence/Species

Based on: `wolf18_vlist.xlsx`

- Virus name field (most shallow taxonomic classification) taken for each record.
- GenBank accession taken from each record.

### Example RdRp

```
rdrp5.Hantaviridae.Bowe_virus:AGW23849.1
rdrp5.Bunyaviridae.Azagny_virus:AEA42011.1
...
rdrp2.Coronaviridae.Night_heron_coronavirus_HKU19:YP_005352862.1
rdrp2.Coronaviridae.Munia_coronavirus_HKU13_3514:YP_002308505.1
rdrp2.Coronaviridae.Wigeon_coronavirus_HKU20:YP_005352870.1
rdrp2.Coronaviridae.Feline_infectious_peritonitis_virus:AGZ84535.1
rdrp2.Coronaviridae.Lucheng_Rn_rat_coronavirus:YP_009336483.1
rdrp2.Coronaviridae.Hipposideros_bat_coronavirus_HKU10:AFU92121.1
rdrp2.Coronaviridae.BtMs_AlphaCoV_GS2013:AIA62270.1
rdrp2.Coronaviridae.Chaerephon_bat_coronavirus_Kenya_KY41_2006:ADX59465.1
rdrp2.Coronaviridae.Porcine_epidemic_diarrhea_virus:AID56804.1
rdrp2.Coronaviridae.Bat_coronavirus_CDPHE15_USA_2006:YP_008439200.1
rdrp2.Coronaviridae.Anlong_Ms_bat_coronavirus:AID16674.1
rdrp2.Coronaviridae.Scotophilus_bat_coronavirus_512:YP_001351683.1
rdrp2.Coronaviridae.BtNv_AlphaCoV_SC2013:YP_009201729.1
rdrp2.Coronaviridae.Bat_coronavirus_1B:ACA52156.1
rdrp2.Coronaviridae.NL63_related_bat_coronavirus:YP_009328933.1
rdrp2.Coronaviridae.NL63_related_bat_coronavirus:APD51489.1
rdrp2.Coronaviridae.229E_related_bat_coronavirus:ALK43115.1
rdrp2.Coronaviridae.Rhinolophus_bat_coronavirus_HKU2:ABQ57223.1
rdrp2.Coronaviridae.Wencheng_Sm_shrew_coronavirus:AID16677.1
rdrp2.Coronaviridae.Human_coronavirus_HKU1:ABD75543.1
rdrp2.Coronaviridae.Betacoronavirus_Erinaceus_VMC_DEU_2012:YP_008719930.1
rdrp2.Coronaviridae.Pipistrellus_bat_coronavirus_HKU5:YP_001039961.1
rdrp2.Coronaviridae.Rousettus_bat_coronavirus:AOG30811.1
rdrp2.Coronaviridae.Rousettus_bat_coronavirus:YP_009273004.1
rdrp2.Coronaviridae.Bat_CoV_279_2005:P0C6V9.1
rdrp2.Coronaviridae.Bat_Hp_betacoronavirus_Zhejiang2013:YP_009072438.1
rdrp2.Coronaviridae.Bottlenose_dolphin_coronavirus_HKU22:AHB63494.1
rdrp2.Coronaviridae.Duck_coronavirus:AKF17722.1
rdrp2.Coronaviridae.Avian_infectious_bronchitis_virus_partridge_GD_S14_2003:AAT70770.1
rdrp2.Coronaviridae.Infectious_bronchitis_virus:ADA83556.1
...
rdrp1.unclassified.Wenzhou_levi_like_virus_3:APG77299.1
rdrp1.Leviviridae.Pseudomonas_phage_PP7:NP_042307.1

```

saved as: `wolf18/gb_assign_group_v2.txt`

In [ ]:
# Parsing Code

# Rename the header to remove virus name
# remove gaps from sequence (unaligned)
sed 's/|.*//g' gb_rdrp.afa \
  | sed 's/-//g' - \
  > rdrp_1.tmp

# One sequence "Pseudomonas_phage_phiYY" has no accession
# YP_009618381.1
sed -i 's/^>$/>YP_009618381.1/g' rdrp_1.tmp

# Iterate through each line / fasta name.
# to swap out headers
while read -r line; do
  # Find headers
  if [[ "$line" = ">"* ]]; then
    acc=$(echo $line | sed 's/\..*//g' - | sed 's/>//g' -)
    newheader=$(grep "$acc" gb_assign_group_v2.txt)
    
    if [[ "$newheader" = "" ]]; then
      echo ">NA_$acc" >> rdrp_2.tmp
    else
      echo $newheader >> rdrp_2.tmp
    fi
    
  else
    # print aa sequence
    echo $line >> rdrp_2.tmp 
  fi
done < rdrp_1.tmp


# Manually remove first ten sequences (Group II introns outgroup)
tail -n +21 rdrp_2.tmp > rdrp_3.tmp

mv rdrp_3.tmp ../wolf18.fa

#rm *.tmp

## Viral RefSeq 240


In [ ]:
# RefSeq (rs) folder
mkdir -p rs; cd rs

wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.protein.faa.gz
wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.2.protein.faa.gz
gzip -d *.gz

cat viral.1.protein.faa viral.2.protein.faa > vrs240.fa
rm viral*faa

In [ ]:
## Run Diamond with ULTRA-SENSITIVE mode
QUERY='vrs240.fa'
RDRP='wolf18'
OUTNAME='vrs240_w18'

diamond makedb --in $RDRP.fa -d $RDRP

# Diamond blastp alignment
time cat $QUERY |\
diamond blastp \
  -d "$RDRP".dmnd \
  --unal 0 \
  --masking 0 \
  --ultra-sensitive \
  -k 1 \
  -p 16 \
  -b 1 \
  -f 6 qseqid qstart qend qlen qstrand \
       sseqid sstart send slen \
       pident evalue cigar \
       qseq \
  > "$OUTNAME".pro


# Fasta of local diamond matches
cut -f1,2,3,4,6,7,8,9,10,13 $OUTNAME.pro \
  | sed 's/^/>/g' \
  | sed 's/\(\t\)\([A-Z]*$\)/\n\2/g' - \
  > $OUTNAME.fa

In [ ]:
# Extract High confidence set of matches
GITB='gitb'

# Extract only matches that are:
# >88% identity to rdrp0
# >98% coverage of rdrp0

cut -f1,2,3,4,6,7,8,9,10,13 $OUTNAME.pro \
  > pro.tmp

rm $GITB.pro

while read -r line; do
  
  pctid=$(echo $line | cut -f 9 -d' ' - )  
  
  subj_start=$(echo $line | cut -f 6 -d' ' - )
  subj_end=$(echo $line | cut -f 7 -d' ' - )
  subj_len=$(echo $line | cut -f 8 -d' ' - )
  
  subj_cov=$( echo "$subj_end - $subj_start + 1" | bc )
  subj_covpct=$( echo "scale=2;$subj_cov / $subj_len" | bc )
  
  if [[ $pctid > 0.88 ]] && [[ $subj_covpct > 0.98 ]]; then
    echo $line >> $GITB.pro
  fi
  
done < pro.tmp
rm pro.tmp


In [ ]:
# Parse GITB set into fasta file
# Fields:
# Query_accession rdrp_start rdrp_end rdrp_len
cut -f1,2,3,10 -d' ' $GITB.pro \
  | sed 's/^/>/g' \
  | sed 's/\( \)\([A-Z]*$\)/\n\2/g' - \
  > $GITB.rdrp.fa

# rdrp sequences in GITB as bed3
cut -f1,2,3 -d' ' $GITB.pro \
  | sed 's/ /\t/g' - \
  > $GITB.rdrp.bed

# From GITB, retrieve full-length records from RefSeq
# (complete ORFs)
cut -f1 $GITB.rdrp.bed | sed 's/>//g' - > $GITB.hits
seqkit grep -r -f $GITB.hits vrs240.fa > $GITB.fa


## Cross-Validation Sets of GITB

In [ ]:
# Create Cross-Validation Set
mkdir cvi; cd cvi
GITB="gitb"

# High-Confidence RDRP only
ln -s ../gitb.rdrp.fa ./$GITB.rdrp.fa

# USEARCH CVI
usearch -threads 30 -calc_distmx $GITB.rdrp.fa -maxdist 1 -termdist 1 -tabbedout gitb.distmat

# 90% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.05 -maxdist 0.15 \
  -tabbedout $GITB.90.cvi

# 75% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.2 -maxdist 0.3 \
  -tabbedout $GITB.75.cvi

# 65% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.3 -maxdist 0.4 \
  -tabbedout $GITB.65.cvi

# 55% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.4 -maxdist 0.5 \
  -tabbedout $GITB.55.cvi

# 45% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.5 -maxdist 0.6 \
  -tabbedout $GITB.45.cvi
  
# 35% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.6 -maxdist 0.7 \
  -tabbedout $GITB.35.cvi

# 25% CVI 
usearch -distmx_split_identity gitb.distmat -mindist 0.7 -maxdist 0.8 \
  -tabbedout $GITB.25.cvi
  
#1 Subset name.
#2 Label1
#3 Label2
#4 Dist

# gitb-decoy set (v0)

Annotated set of non RdRp containing proteins as a negative control

### Human Proteome
File: `homo.fa`
UNIPROT: [UP000005640](https://www.uniprot.org/proteomes/UP000005640)
Accessed: `201230`
Results: `548 843`

### Yeast Proteome
File: `yeast.fa`
UNIPROT: [UP000002311](https://www.uniprot.org/proteomes/UP000002311)
Accessed: `201230`
Results: `57 973`

### Ecoli Proteome
File: `ecoli.fa`
UNIPROT: [UP000000558](https://www.uniprot.org/proteomes/UP000000558)
Accessed: `201230`
Results: `33 586`

### Monodnaviria Proteome (ssDNA viruses)
File: `gb_dsDNA.seq` # Only downloaded accession list
GenBank Query: `txid2731342[Organism:exp]`
Results: `237452`

### Duplodnaviria Proteome (dsDNA viruses)
File: `gb_ssDNA.seq`
GenBank Query: `txid2731341[Organism:exp]`
Results: `1813000`

### Varidnaviria Proteome (vDNA viruses)
File: `vDNA.fa`
GenBank Query: `txid2732004[Organism:exp]`
Accessed: `201230`
Results: `376693`

### Artverviricota  Proteome (Retroviruses)
File: `rv.fa`
GenBank Query: `txid2732409[Organism:exp] `
Accessed: `210105`
Results: `531300`

### Reverse Transcriptase PFAM 
File: `rt_pfam.id97.fa` 
Query: From RCE, 97% clustered
Accessed: `210106`
Results: `58913`


In [ ]:
# create folder for single proteomes
mkdir proteome
mv rv.fa proteome/
mv vdna.fa proteome/
mv homo.fa proteome/
mv yeast.fa proteome/
mv ecoli.fa proteome/

### make gitb-decoy.fa

# Extract DNA virus accessions from genbank v240
cat gb_dsDNA.seq gb_ssDNA.seq gb_vDNA.seq > vdna.acc ; rm *seq
seqkit grep -j 30 -f vdna.acc aaViro_gb240.fa > vdna.tmp
cat rv.fa vdna.tmp > vdna.fa # RV added later

cat vdna.fa homo.fa yeast.fa ecoli.fa rt_pfam.id97.fa \
  | seqkit rmdup -s - \
  | seqkit seq -m 15 \
  > gitb-decoy.fa
  
# Deplete mis-annotations (see below)
echo -e "CZQ50745.1\nAFH02745.1\nAFH02746.1\nADO67072.1\npdb|4TN2|A\nYP_009506261.1\nADE61677.1\nYP_009551602.1\nAXY66749.1\nAWS06671.1" > misannotations.tmp

seqkit grep -j 30 -v -f misannotations.tmp gitb-decoy.fa > gitb-decoy.fa2
rm misannotations.tmp; mv gitb-decoy.fa2 gitb-decoy.fa

seqkit faidx -f gitb-decoy.fa
md5sum gitb-decoy.fa > gitb-decoy.fa.md5

#### Manual clean-up of misannotations

- `CZQ50745.1` Pepper leafroll virus is annotated as a ssDNA virus [with an rdrp](https://apsjournals.apsnet.org/doi/pdf/10.1094/PDIS-03-17-0418-RE). Manually remove accession.
```
CZQ50745.1      655     1085    1085    +       rdrp2.Luteoviridae.Pepper_vein_yellows_virus:AKS03434   2       432     432     96.1    3.9e-301      840.1    431M    VGPQAEVTSLTLQAERWLQRAQSAKIPSSEDRERVINKTVKAYSNVKTFGPTATRGNKLEWRQFLEDFKSAVFSLELDAGIGVPYIAYGRPTHKGWVEDPKLLPVLARLTFNRLQKMLEVESTDMSAEELVQAGLCDPIRTFVKREPHKQSKLDEGRYRLIMSVSLVDQLVARVLFQNQNKREIALWRANPSKPGFGLSTDEQVLEFVQALAAQVEVPPEELITSWKKYLVPTDCSGFDWSVAEWMLHDDMVVRNKLTLDLNPTTEKLRSAWLKCICNSVLCLSDGTLLAQRVPGVQKSGSYNTSSSNSRIRVMAAYHCGADWAMAMGDDALESVNTNLEVYKSLGFKVEVSGQLEFCSHIFRAPDLALPVNERKMLYKLIFGYNPGCGNLEVITNYIAACVSVLNELRHDPDSVALLHTWLVSPVLPQNN
```

- `AFH02745.1`, `AFH02746.1` and `ADO67072.1`: is annotated as a Reverse Transcriptase from a transposable element in a piccovirus (not picorna like in the header). InterPro predicted an RDRP_1 at coordintes 14-91, this is a mis-annotation, removal justified. also: 

```
AFH02746.1      1       89      95      +       rdrp2.unclassified.Hubei_arthropod_virus_1:YP_009336629 116     204     478     67.4    2.4e-35 122.9 89M      KEQLRKEGIVPETIFVDTLKDERKRSDKIKKQGATRVFCASPVDYTIALRQSMLHFCAAYMKNRHTQMHAVGIDVHAEEWARLWSRLTK
```

```
>AFH02745.1 polyprotein, partial [Bat feces associated picorna-like virus SC2797]
QCEEVDHSSLEGAKFIFDEECQVEYIGTVPPNLQPHIPQKTKIEKSLLHTRFGFKSMTEPTILSPQDPRY
THELSPLYYGARKHGIKTKDFTSTMVKRAKEALWTRWLSGMEPIVVDPKKLTPEEAVLGFVGNKYYEALS
LNTSAGYPWSLGGQTTKDSWIETNWEQRTCVIHPQLLKEIQRKEQLRKEGI
>AFH02746.1 polyprotein, partial [Bat feces associated picorna-like virus SC3066]
KEQLRKEGIVPETIFVDTLKDERKRSDKIKKQGATRVFCASPVDYTIALRQSMLHFCAAYMKNRHTQMHA
VGIDVHAEEWARLWSRLTKHGYKFI

```
----

- In the rv-decoy set (a later addition)
- `pdb|4TN2|A`, `YP_009506261.1`, `ADE61677.1`, `YP_009551602.1`, `AXY66749.1`, and `AWS06671.1

----
- `A0A0B4J238` T cell receptor alpha variable 1-2 Protein. IG-like domain, this is the highest scoring False Positive match at E-value of 4.3e-07

```
sp|A0A0B4J238|TVA12_HUMAN       9       105     106     +       rdrp5.unclassified.Zirqa_virus:AMT75437 344     450     660     29.9    4.3e-07 44.3  18M2D10M1D45M7D24M       VSMKMGGTTGQNIDQPTEMTATEGAIVQINCTYQTSGFNGLFWYQQHAGEAPTFLSYNVLDGLEEKGRFSSFLSRSKGYSYLLLKELQMKDSASYLC
```

---

In [ ]:
## Run Diamond with ULTRA-SENSITIVE mode
QUERY='gitb-decoy.fa'
RDRP='wolf18'
OUTNAME='gitb-decoy_w18'

cp ../$RDRP.fa ./
diamond makedb --in $RDRP.fa -d $RDRP

# Diamond blastp alignment
time cat $QUERY |\
diamond blastp \
  -d "$RDRP".dmnd \
  --unal 0 \
  --masking 0 \
  --ultra-sensitive \
  -e 0.05 \
  -k 1 \
  -p 16 \
  -b 1 \
  -f 6 qseqid qstart qend qlen qstrand \
       sseqid sstart send slen \
       pident evalue bitscore \
       cigar  qseq \
  > "$OUTNAME".pro
  
# wc -l
# 7478 / 349304 false-positive hits in decoy set

In [ ]:
aws s3 sync ./ s3://serratus-public/notebook/201230_gitb/decoy/

# Biological Cross Validation

A fairly simple experiment. Using the `wolf18` rdrp-set as a database, compare the distribution matches froma real set of novel rdrp in `wolf20` vs the `gitb-decoy.fa` and `gitb-yoced.fa` (simulated noise from reverse-decoy) set. In contrast to leave-one-out validation, this probably captures what the SRA will look like for diamond quite well.



In [ ]:
mkdir bcv

ln -s ../wolf18.fa ./
aws s3 cp s3://serratus-public/notebook/201210_rdrp0/yaRdRp_201212.fa ./wolf20.fa
ln -s ../decoy/gitb-decoy.fa ./
seqkit seq -r gitb-decoy.fa > gitb-yoced.fa


In [ ]:
## Run Diamond with ULTRA-SENSITIVE mode

run_diamond () {
    QUERY="$1"
    RDRP="$2"
    OUTNAME="$3"

    diamond makedb --in $RDRP.fa -d $RDRP

    # Diamond blastp alignment
    time cat $QUERY |\
    diamond blastp \
      -d "$RDRP".dmnd \
      --unal 0 \
      --masking 0 \
      --ultra-sensitive \
      -e 0.05 \
      -k 1 \
      -p 16 \
      -b 1 \
      -f 6 qseqid qstart qend qlen qstrand \
           sseqid sstart send slen \
           pident evalue bitscore \
           cigar  qseq \
      > "$OUTNAME".pro
}

# Diamond Benchmarks
run_diamond 'wolf20.fa' 'wolf18' 'w20_w18'
run_diamond 'gitb-decoy.fa' 'wolf18' 'decoy_w18'
run_diamond 'gitb-yoced.fa' 'wolf18' 'yoced_w18'

# upload
rm *.fa *.dmnd
aws s3 sync ./ s3://serratus-public/notebook/201230_gitb/bcv/

In [ ]:
## Run Diamond with ULTRA-SENSITIVE - PAM250

run_diamond () {
    QUERY="$1"
    RDRP="$2"
    OUTNAME="$3"

    diamond makedb --in $RDRP.fa -d $RDRP

    # Diamond blastp alignment
    time cat $QUERY |\
    diamond blastp \
      -d "$RDRP".dmnd \
      --unal 0 \
      --masking 0 \
      --ultra-sensitive \
      --matrix PAM250 \
      -e 0.05 \
      -k 1 \
      -p 16 \
      -b 1 \
      -f 6 qseqid qstart qend qlen qstrand \
           sseqid sstart send slen \
           pident evalue bitscore \
           cigar  qseq \
      > "$OUTNAME".pro
}

# Diamond Benchmarks
run_diamond 'wolf20.fa' 'wolf18' 'w20_w18_pam250'
run_diamond 'gitb-decoy.fa' 'wolf18' 'decoy_w18_pam250'
run_diamond 'gitb-yoced.fa' 'wolf18' 'yoced_w18_pam250'

mkdir pam250
mv *_pam250.pro pam250/

# upload
rm *.fa *.dmnd
aws s3 sync ./ s3://serratus-public/notebook/201230_gitb/bcv/

# Target Optimized Diamond (buchfunk)

> `diamond blastx -q query.fna -d rdrp0 -o out -c1 -p1 -k1 --more-sensitive --log --min-orf 1 --masking 0 --gapped-filter-evalue 0 --target-indexed`
>
> `bb`: The optimization is based on fitting a hash set for the database seeds into the cache, so it may suddenly stop working if the database gets bigger. I would probably in any case recommend using a clustered database (choose id% so you don't lose sensitivity), then do a second round of diamond of the aligned reads against the full database.



In [ ]:
# Update Diamond

In [ ]:
cd gitb/bcv

ln -s ../wolf18.fa ./
aws s3 cp s3://serratus-public/notebook/201210_rdrp0/yaRdRp_201212.fa ./wolf20.fa
ln -s ../decoy/gitb-decoy.fa ./
seqkit seq -r gitb-decoy.fa > gitb-yoced.fa

In [ ]:
## Run Diamond with/without buchfink opt

run_diamond () {
    QUERY="$1"
    RDRP="$2"
    OUTNAME="$3"

    diamond makedb --in $RDRP.fa -d $RDRP

# Diamond blastp alignment
    time cat $QUERY |\
    diamond blastp \
        -d "$RDRP".dmnd \
        -c1 -p1 -k1 -e 0.05 \
        --more-sensitive \
        --masking 0 \
        --gapped-filter-evalue 0 \
        --target-indexed \
        > "$OUTNAME".pro
}

# Diamond Benchmarks
run_diamond 'wolf20.fa' 'wolf18' 'w20_w18_bb'
run_diamond 'gitb-decoy.fa' 'wolf18' 'decoy_w18_bb'
run_diamond 'gitb-yoced.fa' 'wolf18' 'yoced_w18_bb'

mkdir pam250
mv *_pam250.pro pam250/
